In [5]:
import pandas as pd
import numpy as np
import os

In [6]:
dataset = pd.read_csv(os.path.join(os.getcwd(),'dataset/train.csv'))
dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [54]:
dataset.describe()
dataset.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [7]:
independent = dataset.drop('label',axis=1)
independent.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [23]:
y = dataset['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [10]:
vocabSize = 10000
messages = independent.copy()
messages.reset_index(inplace=True)
# messages = independent.copy().reset_index(inplace=True)

In [11]:
import nltk,re
from nltk.corpus import stopwords as stw
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Vedang
[nltk_data]     Parasnis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
messages['title']
messages.shape 

(20800, 5)

In [14]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', str(messages['title'][i]))
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stw.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [18]:
onehotRep = [one_hot(words,vocabSize) for words in corpus]
onehotRep

[[2260, 5551, 1012, 496, 8720, 1192, 5581, 4639, 5629, 3815],
 [7861, 2860, 4231, 7873, 7465, 3808, 8848],
 [3133, 8997, 9163, 390],
 [8444, 2409, 3705, 2164, 570, 6431],
 [3300, 7465, 517, 114, 5375, 29, 7465, 1823, 2982, 6706],
 [8669,
  7803,
  6325,
  3024,
  2587,
  162,
  7277,
  7967,
  9655,
  7159,
  2736,
  3023,
  8874,
  1893,
  8848],
 [7350, 7350, 5450, 5249, 6994, 4749, 970, 6993, 3967, 6630, 7953, 5545],
 [1484, 4875, 4045, 2877, 8837, 1746, 3764, 9423, 5137, 4130, 3859],
 [1710, 8454, 5147, 8233, 162, 286, 5286, 7869, 6774, 2502, 5137, 4130, 3859],
 [1534, 1883, 6599, 4754, 4024, 4988, 162, 6700, 5137, 4130, 3859],
 [1604, 7339, 3200, 2847, 9803, 2296, 8962, 3163, 162, 7157],
 [3179, 2013, 7866, 5120, 1561, 811, 1309, 8376],
 [6637, 167, 9150, 9547, 5760, 2324, 3819, 5617, 5097, 2217, 4192],
 [2164, 1455, 8720, 2296, 162, 4024],
 [8054, 9873, 3548, 2866, 8750, 1866, 9968, 1953, 9598],
 [4159, 3505, 9307, 4071, 5017, 9124, 8355, 5137, 4130, 3859],
 [8860, 4268, 5903, 25

In [19]:
### add padding to embedded sequence 
sentence_length = 30
embeddedStartLayer = pad_sequences(onehotRep,padding='pre',maxlen=sentence_length)
print(embeddedStartLayer)

[[   0    0    0 ... 4639 5629 3815]
 [   0    0    0 ... 7465 3808 8848]
 [   0    0    0 ... 8997 9163  390]
 ...
 [   0    0    0 ... 5137 4130 3859]
 [   0    0    0 ... 9986 7833 8111]
 [   0    0    0 ... 3472 7524 8413]]


In [20]:
embedding_vectors = 50
model = Sequential()
model.add(Embedding(vocabSize,embedding_vectors,input_length=sentence_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 50)            500000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 560,501
Trainable params: 560,501
Non-trainable params: 0
_________________________________________________________________


In [24]:
embedding_vectors = 50
blstm = Sequential()
blstm.add(Embedding(vocabSize,embedding_vectors,input_length=sentence_length))
blstm.add(Bidirectional(LSTM(100)))
blstm.add(Dense(1,activation='sigmoid'))
blstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
blstm.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 50)            500000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               120800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 621,001
Trainable params: 621,001
Non-trainable params: 0
_________________________________________________________________


In [25]:
from sklearn.model_selection import train_test_split
X_input = np.array(embeddedStartLayer)
y_input = np.array(y)
X_input.shape

(20800, 30)

In [26]:
y_input.shape

(20800,)

In [27]:
X_train,X_test,Y_train,Y_test = train_test_split(X_input,y_input,test_size=0.4,random_state=42)

In [28]:
blstm.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=65)

Train on 12480 samples, validate on 8320 samples
Epoch 1/10
12480/12480 [==============================] - 23s 2ms/sample - loss: 0.3213 - acc: 0.8545 - val_loss: 0.1796 - val_acc: 0.9249
Epoch 2/10
12480/12480 [==============================] - 20s 2ms/sample - loss: 0.1214 - acc: 0.9535 - val_loss: 0.1842 - val_acc: 0.9309
Epoch 3/10
12480/12480 [==============================] - 20s 2ms/sample - loss: 0.0697 - acc: 0.9760 - val_loss: 0.1941 - val_acc: 0.9299
Epoch 4/10
12480/12480 [==============================] - 20s 2ms/sample - loss: 0.0380 - acc: 0.9879 - val_loss: 0.2468 - val_acc: 0.9321
Epoch 5/10
12480/12480 [==============================] - 20s 2ms/sample - loss: 0.0212 - acc: 0.9937 - val_loss: 0.2870 - val_acc: 0.9220
Epoch 6/10
12480/12480 [==============================] - 20s 2ms/sample - loss: 0.0092 - acc: 0.9975 - val_loss: 0.3347 - val_acc: 0.9251
Epoch 7/10
12480/12480 [==============================] - 20s 2ms/sample - loss: 0.0052 - acc: 0.9986 - val_loss: 0.3

In [37]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=65)

Train on 12480 samples, validate on 8320 samples
Epoch 1/10
12480/12480 [==============================] - 15s 1ms/sample - loss: 0.3104 - acc: 0.8709 - val_loss: 0.1782 - val_acc: 0.9261
Epoch 2/10
12480/12480 [==============================] - 14s 1ms/sample - loss: 0.1123 - acc: 0.9572 - val_loss: 0.1759 - val_acc: 0.9310
Epoch 3/10
12480/12480 [==============================] - 15s 1ms/sample - loss: 0.0563 - acc: 0.9812 - val_loss: 0.2114 - val_acc: 0.9306
Epoch 4/10
12480/12480 [==============================] - 15s 1ms/sample - loss: 0.0243 - acc: 0.9930 - val_loss: 0.2628 - val_acc: 0.9248
Epoch 5/10
12480/12480 [==============================] - 14s 1ms/sample - loss: 0.0123 - acc: 0.9965 - val_loss: 0.2931 - val_acc: 0.9202
Epoch 6/10
12480/12480 [==============================] - 14s 1ms/sample - loss: 0.0051 - acc: 0.9990 - val_loss: 0.4063 - val_acc: 0.9278
Epoch 7/10
12480/12480 [==============================] - 14s 1ms/sample - loss: 0.0029 - acc: 0.9992 - val_loss: 0.3

In [29]:
from sklearn.metrics import confusion_matrix

In [31]:
Y_test.shape

(8320,)

In [32]:
X_test.shape

(8320, 30)

In [42]:
y_pred = blstm.predict_classes(X_test)
y_pred

array([[1],
       [1],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [43]:
confusion_matrix(Y_test,y_pred)

array([[3772,  385],
       [ 224, 3939]], dtype=int64)

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,y_pred)

0.9268028846153846

In [39]:
#lstm model 
ypred = model.predict(X_test)
# accuracy_score(Y_test,model.predict(X_test))

In [45]:
ypred

array([[9.9999708e-01],
       [9.9999583e-01],
       [1.1920929e-07],
       ...,
       [9.9999696e-01],
       [6.7228150e-01],
       [9.9998152e-01]], dtype=float32)